Log into HuggingFace to utilize their stuff

In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

Install Diffusers (https://github.com/huggingface/diffusers/tree/main)

In [ ]:
!git clone https://github.com/losangle/diffusers.git
%cd diffusers
!git branch
!pip install .

Install requirements for Textual Inversion

In [ ]:
!pip install accelerate>=0.16.0
!pip install torchvision
!pip install transformers>=4.25.1
!pip install ftfy
!pip install tensorboard
!pip install Jinja2
!pip install numpy

Get into correct directory

In [ ]:
%cd ./examples/textual_inversion

Download Dataset.

In [ ]:
!wget https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_JPEG.zip
!unzip ./ISIC_2020_Training_JPEG.zip
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1bKdhh7eAjZVWVDl5eH5gQdoZwNlgVS2C' -O ./ISIC_2020_Training_GroundTruth.csv

Select which images to train on and generate differently sampled training datasets

In [6]:
import csv
import os
import numpy as np

def extract_image_data():
    images = []
    first = True
    with open('ISIC_2020_Training_GroundTruth.csv') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
          if (first):
            first = False
          else:
            images.append({
                'name': row[0],
                'sex': row[2],
                'age': float(row[3]) if row[3] != '' else -1,
                'race': row[9],
                'cancer_status': row[6]
            })
    return images

def sample_images(images, output_dir, num_images, criteria):
    categories = ['sex', 'race', 'cancer_status']
    categories = [c for c in categories if criteria[c] != 'ANY'] # ignore the 'ANY' criteria

    passing_images = []
    for image in images:
        pass_criteria = all([criteria[n] == image[n] for n in categories])
        pass_criteria = pass_criteria and (criteria['age_lower'] <= image['age'] <= criteria['age_upper'])
        if (pass_criteria): passing_images.append(image['name'])

    image_samples = np.random.choice(passing_images, size=min(len(passing_images), num_images), replace=False)

    os.system(f'rm -rf {output_dir}') # make sure it is empty if we run this multiple times
    os.mkdir(output_dir)

    for image_sample in image_samples:
        source_path = os.path.join(os.getcwd(), 'train', f'{image_sample}.jpg')
        dst_path = os.path.join(os.getcwd(), output_dir, f'{image_sample}.jpg')
        os.system(f'cp {source_path} {dst_path}')

Sample from entire dataset to create training datasets

In [5]:
images = extract_image_data()

# sex : male / female
# race : very_lt / lt2 / lt1 / int2 / int1 / tan2 / tan1 / Dark (lt = light, so very_lt =  very light and lt2 = light 2, int = intermediate, so int1 = intermediate 1)
# cancer_status : malignant / benign

criteria = {
    'sex': 'ANY',
    'race': 'dark',
    'cancer_status': 'malignant',
    'age_lower': 0,
    'age_upper': 100
}

sample_images(images, './malignant_dataset', 15, criteria)

criteria = {
    'sex': 'ANY',
    'race': 'dark',
    'cancer_status': 'benign',
    'age_lower': 0,
    'age_upper': 100
}
sample_images(images, './benign_dataset', 15, criteria)

!ls benign_dataset
!ls malignant_dataset

33127it [00:00, 267109.42it/s]                           


[{'name': 'ISIC_2637011', 'sex': 'male', 'age': 45.0, 'race': 'very_lt', 'cancer_status': 'benign'}, {'name': 'ISIC_0015719', 'sex': 'female', 'age': 45.0, 'race': 'lt1', 'cancer_status': 'benign'}, {'name': 'ISIC_0052212', 'sex': 'female', 'age': 50.0, 'race': 'very_lt', 'cancer_status': 'benign'}, {'name': 'ISIC_0068279', 'sex': 'female', 'age': 45.0, 'race': 'dark', 'cancer_status': 'benign'}, {'name': 'ISIC_0074268', 'sex': 'female', 'age': 55.0, 'race': 'very_lt', 'cancer_status': 'benign'}]


Delete entire dataset to conserve disk space

In [ ]:
os.system(f'rm -rf {os.path.join(os.getcwd(), "train")}')
os.system(f'rm -rf {os.path.join(os.getcwd(), "ISIC_2020_Training_JPEG.zip")}')
!ls

Import what's neccessary (just textual inversion stuff for now)

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

Run Textual Inversion Pipeline. Adjust the parameters as desired based off of the training dataset used and run as many times as desired

In [ ]:
!accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir="./malignant_dataset" \
  --learnable_property="style" \
  --placeholder_token="<malignant>" \
  --initializer_token="melanoma" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=1 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir="textual_inversion_melanoma" \
  --push_to_hub

Use learned embeddings to generate images

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

def generate_images(embedding_dir, num_images, output_dir, generation_string):
    pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
    pipeline.load_textual_inversion(embedding_dir)

    os.system(f'rm -rf {output_dir}')
    os.mkdir(output_dir)

    while (num_images > 0):
        images = pipeline(generation_string, num_inference_steps=50).images
        for image in images:
            image.save(os.path.join(output_dir, f'{num_images}.png'))
            num_images -= 1

Generate images

In [ ]:
generate_images('./textual_inversion_melanoma', 10, './images_out', "melanoma <malignant>")